# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [550]:
# All required libraries are imported here for you.
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from scipy.stats import chi2_contingency
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics

# Load the dataset
crops = pd.read_csv("soil_measures.csv")

# Write your code here
crops

,N,P,K,ph,crop
0,90,42,43,6.502985,rice
1,85,58,41,7.038096,rice
2,60,55,44,7.840207,rice
3,74,35,40,6.980401,rice
4,78,42,42,7.628473,rice
...,...,...,...,...,...
2195,107,34,32,6.780064,coffee
2196,99,15,27,6.086922,coffee
2197,118,33,30,6.362608,coffee
2198,117,32,34,6.758793,coffee


In [551]:
crops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB


In [552]:
crops.isna().sum()
#No hay valores faltantes aparentemente

N       0
P       0
K       0
ph      0
crop    0
dtype: int64

In [553]:
crops.duplicated().sum()
#No hay datos duplicados

0

In [554]:
crops["crop"].nunique()

22

Sabiendo que los datos estan aparentemente limpios, procedemos a hacer una prueba Chi Cuadrada, dado que nuestra variable objetivo es de tipo categorico y nominal.

In [555]:
features = crops.drop(columns=["crop"]).columns.tolist()
features

['N', 'P', 'K', 'ph']

In [556]:
X = crops.drop(columns=["crop"])
y = crops["crop"]
X.shape, y.shape

((2200, 4), (2200,))

In [557]:
X

,N,P,K,ph
0,90,42,43,6.502985
1,85,58,41,7.038096
2,60,55,44,7.840207
3,74,35,40,6.980401
4,78,42,42,7.628473
...,...,...,...,...
2195,107,34,32,6.780064
2196,99,15,27,6.086922
2197,118,33,30,6.362608
2198,117,32,34,6.758793


In [558]:
std_sclr = StandardScaler()
X = pd.DataFrame(std_sclr.fit_transform(X), columns=features)
X

,N,P,K,ph
0,1.068797,-0.344551,-0.101688,0.043302
1,0.933329,0.140616,-0.141185,0.734873
2,0.255986,0.049647,-0.081939,1.771510
3,0.635298,-0.556811,-0.160933,0.660308
4,0.743673,-0.344551,-0.121436,1.497868
...,...,...,...,...
2195,1.529390,-0.587134,-0.318922,0.401395
2196,1.312641,-1.163269,-0.417666,-0.494413
2197,1.827421,-0.617457,-0.358420,-0.138120
2198,1.800327,-0.647780,-0.279425,0.373904


In [559]:
y = pd.get_dummies(y, columns=["crop"])
y

,apple,banana,blackgram,chickpea,coconut,coffee,cotton,grapes,jute,kidneybeans,lentil,maize,mango,mothbeans,mungbean,muskmelon,orange,papaya,pigeonpeas,pomegranate,rice,watermelon
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2195,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2196,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2197,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2198,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [560]:
X.shape, y.shape

((2200, 4), (2200, 22))

In [561]:
for feature in features:
    contingency_table = pd.crosstab(X[feature], crops["crop"].astype("category"))
    chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)
    print(f'Feature: {feature}, p-value: {p_val}')

Feature: N, p-value: 0.0
Feature: P, p-value: 0.0
Feature: K, p-value: 0.0
Feature: ph, p-value: 0.47158596287103616


In [562]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [563]:
rf = RandomForestClassifier(n_estimators=25)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=25)

In [564]:
importances = rf.feature_importances_
importances

array([0.21269351, 0.25911646, 0.32419771, 0.20399232])

In [565]:
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7909090909090909

Segun 

In [566]:
for feature in features:
    x = X[[feature]]
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
    rf = RandomForestClassifier(n_estimators=25)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy for {feature}: {accuracy:.4f}')

Accuracy for N: 0.0273
Accuracy for P: 0.0568
Accuracy for K: 0.1227
Accuracy for ph: 0.1023


In [567]:
encoder = LabelEncoder()
y = encoder.fit_transform(crops["crop"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)
importance = model.coef_[0]
importance

array([-1.20788632,  3.91571831,  4.50350247, -1.18903345])

Como vemos, tanto probando con Random Forest con todas las caracteristicas en conjunto, como por separado, y utilizando Regresion Logistica como sugiere el ejercicio, en todas obtenemos que el Nivel de Potasio (K) es la caracteristica que produce el mejor score para predecir el tipo de cosecha.

In [568]:
best_predictive_feature = {"K":4.5093985}